In [ ]:
from openai import OpenAI
from pydantic import BaseModel,Field
from duckduckgo_search import DDGS
from IPython.display import Markdown
import json

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# search = DuckDuckGoSearchRun()

# search.invoke("What is the Latest news of west bengal")

In [ ]:
api_key = 'your_api_key'
api_key

In [ ]:
#initialize the agent with the api key
client = OpenAI(api_key=api_key)

Model = "gpt-4o-mini"

In [ ]:
def search_for_topic(topic:str):
    try:
        search = DuckDuckGoSearchRun()
        response = search.invoke(topic)
        return response
    except Exception as e:
        return str(e)

In [ ]:
POST_CREATION_PROMPT = """
Make a Plagarism free Post with this following Context:{context}
Make it Interactive and Engaging.
Use Emojis to make it more attractive.
"""

In [ ]:
def Create_post(context:str):
    prompt = POST_CREATION_PROMPT.format(context=context)
    response = client.chat.completions.create(
        model=Model,
        messages=[{"role": "user", "content": prompt}],
    )
    
    return response.choices[0].message.content

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options  # To configure Firefox in headless mode
import time

class LinkedInBot:
    def __init__(self, email: str, password: str, driver_path: str):
        """
        Initialize the LinkedInBot with user credentials and ChromeDriver path.
        
        :param email: User's LinkedIn email/username.
        :param password: User's LinkedIn password.
        :param driver_path: Path to the ChromeDriver executable.
        """
        self.email = email
        self.password = password
        # Set up the ChromeDriver service
        # service = Service(driver_path)
        options = Options()
        options.add_argument("--headless")  # Run in headless mode
        options.add_argument("--disable-gpu")  # Disable GPU for stability in headless mode
        self.driver = webdriver.Chrome()
    
    def login(self):
        """
        Logs into LinkedIn using the provided credentials.
        
        :return: Current URL after attempting login.
        """
        # Open LinkedIn's login page
        self.driver.get("https://www.linkedin.com/login")
        # time.sleep(2)  # Wait for the page to load
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.ID, "organic-div")))
        # Locate the email and password input fields
        email_field = self.driver.find_element(By.ID, "username")
        password_field = self.driver.find_element(By.ID, "password")
        
        # Fill in the login credentials
        email_field.send_keys(self.email)
        password_field.send_keys(self.password)
        
        # Submit the form (simulate pressing 'Enter')
        password_field.send_keys(Keys.RETURN)
        time.sleep(3)  # Allow time for login process
        
        return self.driver.current_url
    
    def post(self, content: str): 
        """
        Posts the given content on the user's LinkedIn feed.
        
        :param content: The text content to be posted.
        """
        # Click on the 'Start a post' button
        start_post_button = self.driver.find_element(By.CSS_SELECTOR, ".artdeco-button.artdeco-button--muted.artdeco-button--4.artdeco-button--tertiary.ember-view.dQcafCkQrDTgDAnkmIqDAVtYuNXxzBUlU")
        self.driver.execute_script("arguments[0].click();", start_post_button)

        # start_post_button.click()
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "share-box")))

        # Locate the text area for typing the post content
        post_content_area = self.driver.find_element(By.CSS_SELECTOR, ".ql-editor.ql-blank")
        post_content_area.send_keys(content)

        # time.sleep(5)  # Wait for the content to be typed
        # Click on the 'Post' button
        post_button = WebDriverWait(self.driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".share-actions__primary-action.artdeco-button.artdeco-button--2.artdeco-button--primary")))
        print("hi")
        self.driver.execute_script("arguments[0].click();", post_button)
        time.sleep(10)  # Allow time for the post to be shared



    def quit(self):
        """
        Closes the browser window.
        """
        self.driver.quit()

In [ ]:
def post_content_on_linkedIn(content:str):
    try:
        user_email = "sayangolder2004@gmail.com"
        user_password = "amijabona@"
        chromedriver_path = "/path/to/chromedriver"  # Update with the actual path

        # Create an instance of the bot and log in
        linkedin_bot = LinkedInBot(user_email, user_password, chromedriver_path)
        current_page = linkedin_bot.login()
        linkedin_bot.post()  # Post a message on the feed
        return "Posted Successfully"
    except Exception as e:
        return str(e)

In [ ]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_for_topic",
            "description": "Look up for the latest news on a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string", "description": "The topic to search for."}
                },
                "required": ["topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "Create_post", 
            "description": "Create a plagiarism-free post based on a given context.",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {"type": "string", "description": "The context to base the post on."}
                },
                "required": ["context"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "post_content_on_linkedIn",
            "description": "Post content on LinkedIn.",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {"type": "string", "description": "The content to post on LinkedIn."}
                },
                "required": ["content"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "search_for_topic": search_for_topic,
    "Create_post": Create_post, 
    "post_content_on_linkedIn": post_content_on_linkedIn
}

In [ ]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):
    
    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that can Search lastest news on topics to create plagarism free linkedin content to post on LinkedIn.
"""

In [ ]:
def run_agent(messages):
    print("Running agent with messages:", messages)

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        print("Making router call to OpenAI")
        response = client.chat.completions.create(
            model=Model,
            messages=messages,
            tools=tools,
        )
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        print("Received response with tool calls:", bool(tool_calls))

        # if the model decides to call function(s), call handle_tool_calls
        if tool_calls:
            print("Processing tool calls")
            messages = handle_tool_calls(tool_calls, messages)
        else:
            print("No tool calls, returning final response")
            return response.choices[0].message.content

In [ ]:
result = run_agent('Make a Post on the Latest GPU Import news in India')